In [320]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, log_loss
from sklearn.model_selection import train_test_split
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier, cv, Pool, CatBoost
import matplotlib.pyplot as plt
import seaborn as sns

Load Data

In [321]:
age_gender_df = pd.read_csv("alter_geschlecht.csv")
interests_df = pd.read_csv("interesse.csv")
rest_df = pd.read_csv("rest.csv")

Check Each dataframe to understand the data better and look for unknown entries

In [322]:
age_gender_null_value_stats = age_gender_df.isnull().sum(axis=0)
age_gender_null_value_stats[age_gender_null_value_stats != 0]

Series([], dtype: int64)

In [323]:
age_gender_df.head()

,id,Geschlecht,Alter
0,170727,Male,42
1,41795,Male,24
2,369321,Female,30
3,500463,Female,32
4,327695,Male,34


In [324]:
age_gender_df.describe()

,id,Alter
count,508146.000000,508146.000000
mean,254073.500000,38.808413
std,146689.259281,15.500179
min,1.000000,20.000000
25%,127037.250000,25.000000
50%,254073.500000,36.000000
75%,381109.750000,49.000000
max,508146.000000,85.000000


In [325]:
interests_null_value_stats = interests_df.isnull().sum(axis=0)
interests_null_value_stats[interests_null_value_stats != 0]

Series([], dtype: int64)

In [326]:
interests_df.head()

,id,Interesse
0,1,1.0
1,2,0.0
2,3,1.0
3,4,0.0
4,5,0.0


In [327]:
interests_df.describe()

,id,Interesse
count,381109.000000,381109.000000
mean,190555.000000,0.122563
std,110016.836208,0.327936
min,1.000000,0.000000
25%,95278.000000,0.000000
50%,190555.000000,0.000000
75%,285832.000000,0.000000
max,381109.000000,1.000000


In [328]:
rest_null_value_stats = rest_df.isnull().sum(axis=0)
rest_null_value_stats[rest_null_value_stats != 0]

Series([], dtype: int64)

In [329]:
rest_df.head()

,Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id
0,1;15.0;1;1-2 Year;No;2630.0;124.0;74;317635
1,1;28.0;0;1-2 Year;Yes;2630.0;125.0;213;337993
2,1;33.0;0;1-2 Year;Yes;27204.0;124.0;114;160325
3,1;46.0;1;< 1 Year;No;31999.0;152.0;251;141620
4,1;49.0;0;1-2 Year;Yes;28262.0;26.0;60;75060


In [330]:
rest_df.describe()

,Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id
count,508146
unique,508146
top,1;15.0;1;1-2 Year;No;2630.0;124.0;74;317635
freq,1


Properly separate features by column

In [331]:
rest_df[["Fahrerlaubnis", "Regional_Code", "Vorversicherung", "Alter_Fzg", "Vorschaden", "Jahresbeitrag", "Vertriebskanal","Kundentreue","id"]]= rest_df['Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id'].str.split(';', n=-1, expand=True)

In [332]:
rest_df = rest_df.drop("Fahrerlaubnis;Regional_Code;Vorversicherung;Alter_Fzg;Vorschaden;Jahresbeitrag;Vertriebskanal;Kundentreue;id", axis=1)
rest_df.head()

,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,id
0,1,15.0,1,1-2 Year,No,2630.0,124.0,74,317635
1,1,28.0,0,1-2 Year,Yes,2630.0,125.0,213,337993
2,1,33.0,0,1-2 Year,Yes,27204.0,124.0,114,160325
3,1,46.0,1,< 1 Year,No,31999.0,152.0,251,141620
4,1,49.0,0,1-2 Year,Yes,28262.0,26.0,60,75060


In [333]:
rest_df.describe()

,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,id
count,508146,508146,508146,508146,508146,508146,508146,508146,508146
unique,2,53,2,3,2,52062,157,290,508146
top,1,28.0,0,1-2 Year,Yes,2630.0,152.0,256,317635
freq,507097,141937,275076,267015,256248,86488,179523,1877,1


Cast id as int to remain consistent

In [334]:
rest_df['id']= rest_df['id'].astype(int)

Merge data into a single frame for easier manipulation

In [335]:
merged_df = pd.merge(age_gender_df, rest_df, on='id', how='outer')
merged_df.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39


In [336]:
merged_df =pd.merge(merged_df, interests_df, on='id', how='outer')

Clean feature formatting (numerical and categorical features)

In [337]:
merged_df['Regional_Code']= merged_df['Regional_Code'].astype(float).astype(int)
merged_df['Vertriebskanal']= merged_df['Vertriebskanal'].astype(float).astype(int)
merged_df.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,Interesse
0,1,Male,44,1,28,0,> 2 Years,Yes,40454.0,26,217,1.0
1,2,Male,76,1,3,0,1-2 Year,No,33536.0,26,183,0.0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294.0,26,27,1.0
3,4,Male,21,1,11,1,< 1 Year,No,28619.0,152,203,0.0
4,5,Female,29,1,41,1,< 1 Year,No,27496.0,152,39,0.0


In [338]:
merged_df_null_value_stats = merged_df.isnull().sum(axis=0)
merged_df_null_value_stats[merged_df_null_value_stats != 0]

Interesse    127037
dtype: int64

In [339]:
unknown_rows= merged_df[merged_df['Interesse'].isna()] #saved as a variable as it could be used for future testing samples

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,Interesse
381109,381110,Male,25,1,11,1,< 1 Year,No,35786.0,152,53,NaN
381110,381111,Male,40,1,28,0,1-2 Year,Yes,33762.0,7,111,NaN
381111,381112,Male,47,1,28,0,1-2 Year,Yes,40050.0,124,199,NaN
381112,381113,Male,24,1,27,1,< 1 Year,Yes,37356.0,152,187,NaN
381113,381114,Male,27,1,28,1,< 1 Year,No,59097.0,152,297,NaN


Remove rows where interest of customer is unknown

In [340]:
merged_df = merged_df[merged_df['Interesse'].notna()]
merged_df_null_value_stats = merged_df.isnull().sum(axis=0)
print(merged_df_null_value_stats[merged_df_null_value_stats != 0])


Series([], dtype: int64)


In [341]:

merged_df['Interesse'] = merged_df['Interesse'].astype(bool)
merged_df.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue,Interesse
0,1,Male,44,1,28,0,> 2 Years,Yes,40454.0,26,217,True
1,2,Male,76,1,3,0,1-2 Year,No,33536.0,26,183,False
2,3,Male,47,1,28,0,> 2 Years,Yes,38294.0,26,27,True
3,4,Male,21,1,11,1,< 1 Year,No,28619.0,152,203,False
4,5,Female,29,1,41,1,< 1 Year,No,27496.0,152,39,False


In [342]:
merged_df['Alter_Fzg'].unique()

array(['> 2 Years', '1-2 Year', '< 1 Year'], dtype=object)

Create a pipeline that handles the One Hot encoding of categorical features and uses a decision tree classifier as estimator

In [343]:
merged_df['Kundentreue']= merged_df['Kundentreue'].astype(int)
merged_df['Jahresbeitrag']= merged_df['Jahresbeitrag'].astype(float)
merged_df['Fahrerlaubnis']= merged_df['Fahrerlaubnis'].astype(bool)
merged_df['Vorversicherung']= merged_df['Vorversicherung'].astype(bool)

In [344]:
categorical_cols =['Geschlecht', 'Alter_Fzg','Vorschaden']

categorical_pipe = Pipeline(steps=[
                            ('oh_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
dt_transformer = ColumnTransformer(transformers =[                
                                        ('categorical_pipe', categorical_pipe, categorical_cols)
],
                                        remainder = 'drop',
                                        n_jobs =-1)
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier_pipeline = make_pipeline(dt_transformer, dt_classifier)

In [345]:
X.columns

Index(['id', 'Geschlecht', 'Alter', 'Fahrerlaubnis', 'Regional_Code',
       'Vorversicherung', 'Alter_Fzg', 'Vorschaden', 'Jahresbeitrag',
       'Vertriebskanal', 'Kundentreue'],
      dtype='object')

In [346]:
X.head()

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue
0,1,Male,44,True,28,True,> 2 Years,Yes,40454.0,26,217
1,2,Male,76,True,3,True,1-2 Year,No,33536.0,26,183
2,3,Male,47,True,28,True,> 2 Years,Yes,38294.0,26,27
3,4,Male,21,True,11,True,< 1 Year,No,28619.0,152,203
4,5,Female,29,True,41,True,< 1 Year,No,27496.0,152,39


In [347]:
X = merged_df.drop(['id', 'Interesse'],axis=1)
y= merged_df['Interesse']

Perform stratified cross validation for balanced class representation and investigate log loss (metric commonly used for binary classification tasks) of model

In [348]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#dt_classifier_scores= 
dt_cv=cross_validate(dt_classifier_pipeline,X,y,cv=skf,verbose=1,return_train_score=True,return_estimator=True,scoring = ['neg_log_loss']) #'accuracy'])


In [349]:
print(f"Train log loss {dt_cv['train_neg_log_loss']*-1}")
print(f"Test log loss{dt_cv['test_neg_log_loss']*-1}")

#print(f"Train accuracy {dt_cv['train_accuracy']}")
#print(f"Test accuracy{dt_cv['test_accuracy']}")

Train log loss [0.28747159 0.28786419 0.28756807 0.28777364 0.28776114]
Test log loss[0.2886002  0.28701367 0.28819829 0.28736682 0.2874206 ]


In [350]:
params = {
    'loss_function': 'Logloss',
    'iterations': 100,
    'learning_rate': 0.5,
}

Helper method for easier visualisation of cross validation results

In [351]:
def print_cv_summary(cv_data):
    cv_data.head(10)

    best_value = cv_data['test-Logloss-mean'].min()
    best_iter = cv_data['test-Logloss-mean'].values.argmin()

    print('Best validation Logloss score : {:.4f}±{:.4f} on step {}'.format(
        best_value,
        cv_data['test-Logloss-std'][best_iter],
        best_iter)
    )

Format data as Pool for using with CatBoost models

In [352]:
cat_pool = Pool(data=X,label =y,cat_features=categorical_cols)

In [353]:
catboost_cv_data, fitted_cat_model = cv(
    params = params,
    pool = cat_pool,
    fold_count=4,
    shuffle=True,
    partition_random_seed=0,
    plot=False,
    stratified=True,
    verbose=False,
    early_stopping_rounds=10,
    return_models = True
    
)

print_cv_summary(catboost_cv_data)   #Overkill for this scenario

Training on fold [0/4]

bestTest = 0.2734418645
bestIteration = 35

Training on fold [1/4]

bestTest = 0.2738243951
bestIteration = 35

Training on fold [2/4]

bestTest = 0.2734263256
bestIteration = 35

Training on fold [3/4]

bestTest = 0.272956044
bestIteration = 36

Best validation Logloss score : 0.2734±0.0004 on step 35


In [354]:
classifier = CatBoostClassifier(iterations=200)
classifier.fit(cat_pool)

Learning rate set to 0.5
0:	learn: 0.3266568	total: 546ms	remaining: 1m 48s
1:	learn: 0.2884756	total: 1.13s	remaining: 1m 52s
2:	learn: 0.2815143	total: 1.68s	remaining: 1m 49s
3:	learn: 0.2783545	total: 2.33s	remaining: 1m 54s
4:	learn: 0.2768457	total: 3.43s	remaining: 2m 13s
5:	learn: 0.2759500	total: 4.11s	remaining: 2m 13s
6:	learn: 0.2755964	total: 4.71s	remaining: 2m 10s
7:	learn: 0.2752852	total: 5.31s	remaining: 2m 7s
8:	learn: 0.2750656	total: 6.04s	remaining: 2m 8s
9:	learn: 0.2744261	total: 6.65s	remaining: 2m 6s
10:	learn: 0.2742662	total: 7.23s	remaining: 2m 4s
11:	learn: 0.2741405	total: 7.93s	remaining: 2m 4s
12:	learn: 0.2739896	total: 8.5s	remaining: 2m 2s
13:	learn: 0.2737860	total: 9.05s	remaining: 2m
14:	learn: 0.2735994	total: 9.54s	remaining: 1m 57s
15:	learn: 0.2735532	total: 10.2s	remaining: 1m 56s
16:	learn: 0.2733810	total: 10.7s	remaining: 1m 55s
17:	learn: 0.2733043	total: 11.3s	remaining: 1m 54s
18:	learn: 0.2731792	total: 11.9s	remaining: 1m 53s
19:	lear

In [356]:
dt_classifier_pipeline.score(X,y)

0.877436638861848

In [358]:
classifier.score(X,y)

0.8804567722095253

Save the (slightly superior in comparison to the decision tree classifier) catboost model

In [359]:
classifier.save_model("catmodel",
           format="cbm",
           export_parameters=None,
           pool=cat_pool)